
# One‑Class SVM (OC‑SVM) — AIS Anomaly Detection (Galápagos)

**Objetivo:** Entrenar y evaluar un detector no supervisado (One‑Class SVM, kernel RBF) para identificar anomalías en series de tiempo de trayectorias AIS de pesqueros alrededor de la RMG, usando **solo comportamiento normal** para el entrenamiento y validando contra puntos/ventanas marcadas como `is_suspicious = 1`.

**Pautas (resumen de la planificación):**
- Entrenar **solo con `is_suspicious = 0`** (comportamiento normal).
- Ingeniería de variables por MMSI y orden temporal: lags, diferencias, SMA/EMA, distancia Haversine por segmento, delta de tiempo, aceleración, velocidad angular, y ratios como `distancia_a_costa/distancia_a_puerto`.
- Serialización por **ventanas deslizantes** (tamaño `T`) y representación vectorizada `(N, T×F)` para modelos tabulares.
- **Split por MMSI** (GroupKFold) para evitar fuga por identidad.
- Métricas: ROC‑AUC, PR‑AUC, Precision@k, Recall@k, F1@k; análisis por ventana y por trayectoria.
- Guardado de **artefactos**: scaler, modelo, configuración y scores para replicabilidad.


In [1]:

# --- Setup ---
import os, math, json, gc, pickle, warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.model_selection import GroupKFold, ParameterGrid
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, auc

import matplotlib.pyplot as plt

# Reproducibilidad
import random
SEED = 42
random.seed(SEED); np.random.seed(SEED)

# --- Configuración global ---
CFG = {
    # Entradas (ajusta paths a tu estructura real)
    "input_parquet": "/teamspace/studios/profound-silver-kn8tf/ais_anomaly/data/ais_enriched.parquet",        # Dataset enriquecido (AIS + contexto)
    "train_filter_col": "is_suspicious",                  # Col de validación (no usada para entrenar)
    "mmsi_col": "mmsi",
    "timestamp_col": "timestamp",
    # Variables base esperadas en el parquet (ajusta a tus nombres)
    "base_features": ["lat", "lon", "speed", "course", "depth", "dist_coast_km", "dist_port_km"],
    # Parámetros de ingeniería
    "lags": [1, 2],
    "ema_spans": [3, 5],
    "window_size": 10,    # T
    "step_size": 5,
    # Modelado
    "svm_nu_grid": [0.01, 0.05, 0.1],
    "svm_gamma_grid": ["scale", 0.1, 0.01],
    "kernel": "rbf",
    # Evaluación
    "kfold_splits": 5,
    # Salidas
    "out_dir": "runs/ocsvm",
    "artifact_prefix": "ocsvm_rbf"
}

os.makedirs(CFG["out_dir"], exist_ok=True)
print("Config:", json.dumps(CFG, indent=2))


Config: {
  "input_parquet": "/teamspace/studios/profound-silver-kn8tf/ais_anomaly/data/ais_enriched.parquet",
  "train_filter_col": "is_suspicious",
  "mmsi_col": "mmsi",
  "timestamp_col": "timestamp",
  "base_features": [
    "lat",
    "lon",
    "speed",
    "course",
    "depth",
    "dist_coast_km",
    "dist_port_km"
  ],
  "lags": [
    1,
    2
  ],
  "ema_spans": [
    3,
    5
  ],
  "window_size": 10,
  "step_size": 5,
  "svm_nu_grid": [
    0.01,
    0.05,
    0.1
  ],
  "svm_gamma_grid": [
    "scale",
    0.1,
    0.01
  ],
  "kernel": "rbf",
  "kfold_splits": 5,
  "out_dir": "runs/ocsvm",
  "artifact_prefix": "ocsvm_rbf"
}


In [2]:

from math import radians, cos, sin, asin, sqrt

def haversine_km(lat1, lon1, lat2, lon2):
    # Coordinates in decimal degrees
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def angular_diff_deg(a, b):
    # minimal signed angle difference in degrees
    diff = (a - b + 180) % 360 - 180
    return diff

def ensure_datetime(df, col):
    if not np.issubdtype(df[col].dtype, np.datetime64):
        df[col] = pd.to_datetime(df[col], utc=True, errors="coerce")
    return df

def per_mmsi_sort(df, mmsi_col, ts_col):
    return df.sort_values([mmsi_col, ts_col]).reset_index(drop=True)

def add_temporal_dynamics(df, mmsi_col, ts_col):
    # Requires sorted df
    df = df.copy()
    # segment distance (km) between consecutive points
    df["lat_prev"] = df.groupby(mmsi_col)["lat"].shift(1)
    df["lon_prev"] = df.groupby(mmsi_col)["lon"].shift(1)
    df["segment_km"] = haversine_km(df["lat_prev"], df["lon_prev"], df["lat"], df["lon"])
    df["segment_km"] = df["segment_km"].fillna(0.0)

    # delta time (seconds)
    df["t_prev"] = df.groupby(mmsi_col)[ts_col].shift(1)
    dt = (df[ts_col] - df["t_prev"]).dt.total_seconds()
    df["delta_t_s"] = dt.fillna(0.0)

    # speed (knots) -> m/s conversion if needed; assume 'speed' es nudo; 1 knot = 0.514444 m/s
    # Si tu columna ya está en m/s, ajusta esta sección.
    spd_ms = df["speed"].astype(float) * 0.514444
    df["acc_ms2"] = (spd_ms - spd_ms.groupby(df[mmsi_col]).shift(1)) / df["delta_t_s"].replace(0, np.nan)
    df["acc_ms2"] = df["acc_ms2"].fillna(0.0)

    # angular velocity (deg/s) con diferencia mínima de ángulo
    df["course_prev"] = df.groupby(mmsi_col)["course"].shift(1)
    dtheta = angular_diff_deg(df["course"], df["course_prev"])
    df["ang_vel_deg_s"] = (dtheta / df["delta_t_s"].replace(0, np.nan)).fillna(0.0)

    # ratios robustos
    df["ratio_coast_port"] = df["dist_coast_km"] / (df["dist_port_km"].replace(0, np.nan))
    df["ratio_coast_port"] = df["ratio_coast_port"].replace([np.inf, -np.inf], np.nan).fillna(0.0)

    df.drop(columns=["lat_prev", "lon_prev", "t_prev", "course_prev"], inplace=True)
    return df

def add_lags_ema(df, mmsi_col, cols, lags=[1,2], ema_spans=[3,5]):
    df = df.copy()
    for c in cols:
        for L in lags:
            df[f"{c}_lag{L}"] = df.groupby(mmsi_col)[c].shift(L)
        for span in ema_spans:
            df[f"{c}_ema{span}"] = df.groupby(mmsi_col)[c].transform(lambda s: s.ewm(span=span, adjust=False).mean())
    return df


In [3]:

def build_windows(df, mmsi_col, ts_col, feature_cols, T=10, step=5):
    # df sorted per mmsi, ts; we assume it
    X = []
    groups = []
    idx_rows = []  # track terminal row index per window if useful
    by_mmsi = df.groupby(mmsi_col, sort=False)
    for mmsi, g in by_mmsi:
        g = g.reset_index(drop=True)
        n = len(g)
        for start in range(0, max(0, n - T + 1), step):
            end = start + T
            if end > n: break
            win = g.iloc[start:end]
            X.append(win[feature_cols].to_numpy().reshape(-1))  # flatten (T*F)
            groups.append(mmsi)
            idx_rows.append(win.index[-1])
    X = np.array(X, dtype=np.float32) if len(X) else np.empty((0, len(feature_cols)*T), dtype=np.float32)
    return X, np.array(groups), np.array(idx_rows)


In [ ]:
# --- Carga de datos ---
from pathlib import Path
import glob

def resolve_input_parquet(cfg):
    """Resuelve la ruta real del parquet de entrada.
    - Primero intenta cfg["input_parquet"].
    - Luego intenta candidatos comunes.
    - Finalmente hace un glob recursivo buscando archivos .parquet con nombres típicos.
    """
    p = Path(cfg["input_parquet"])
    if p.exists():
        return str(p)

    candidates = [
        Path("/teamspace/studios/profound-silver-kn8tf/ais_anomaly/data"),
        Path("data/ais_enriched.parquet"),
        Path("/mnt/data/ais_enriched.parquet"),
        Path("/workspace/data/ais_enriched.parquet"),
        Path("/workspace/ais_enriched.parquet"),
        Path(os.environ.get("AIS_PARQUET", "")) if os.environ.get("AIS_PARQUET") else None,
    ]
    for c in candidates:
        if c and c.exists():
            return str(c)

    # Búsqueda amplia (recursiva). Filtra por nombres comunes para evitar abrir algo enorme por error.
    hits = [Path(h) for h in glob.glob("**/*.parquet", recursive=True)]
    name_whitelist = {"ais_enriched.parquet", "ais_clean.parquet", "ais_windows.parquet"}
    prio = [h for h in hits if h.name in name_whitelist] or hits

    if prio:
        print("[resolve_input_parquet] Usando:", prio[0])
        return str(prio[0])

    raise FileNotFoundError(
        "No se encontró el parquet de entrada. "
        "Actualiza CFG['input_parquet'] con la ruta correcta o define AIS_PARQUET en el entorno."
    )

CFG["input_parquet"] = resolve_input_parquet(CFG)
print("Input parquet:", CFG["input_parquet"])

df = pd.read_parquet(CFG["input_parquet"])

# Asegurar tipos y orden
df = ensure_datetime(df, CFG["timestamp_col"])
df = per_mmsi_sort(df, CFG["mmsi_col"], CFG["timestamp_col"])

# Ingeniería de variables dinámicas
df = add_temporal_dynamics(df, CFG["mmsi_col"], CFG["timestamp_col"])

# Lags y EMAs sobre subconjunto de columnas continuas relevantes
cont_cols = ["speed", "course", "depth", "segment_km", "delta_t_s", "acc_ms2", "ang_vel_deg_s", "dist_coast_km", "dist_port_km", "ratio_coast_port"]
df = add_lags_ema(df, CFG["mmsi_col"], cont_cols, lags=CFG["lags"], ema_spans=CFG["ema_spans"])

# Selección de features finales (excluye etiquetas/ids)
feature_cols = [c for c in df.columns if c not in [CFG["train_filter_col"], CFG["mmsi_col"], CFG["timestamp_col"], "lat", "lon"]]
print("N features:", len(feature_cols))
print("Ejemplo de features:", feature_cols[:20])

# Split por etiqueta (solo para EVALUACIÓN; entrenamiento NO usa la etiqueta)
mask_norm = (df[CFG["train_filter_col"]] == 0)
mask_susp = (df[CFG["train_filter_col"]] == 1)

print("Puntos normales:", int(mask_norm.sum()))
print("Puntos sospechosos:", int(mask_susp.sum()))

Input parquet: /teamspace/studios/profound-silver-kn8tf/ais_anomaly/data


In [ ]:

T = CFG["window_size"]; step = CFG["step_size"]

# Construimos ventanas en TODO el dataset — etiquetamos por la proporción de puntos sospechosos dentro de la ventana
# Para entrenamiento, luego filtramos a ventanas completamente normales (o con umbral bajo de sospechosos)
X_all, groups_all, idx_rows_all = build_windows(df, CFG["mmsi_col"], CFG["timestamp_col"], feature_cols, T, step)

# Derivamos una etiqueta de ventana basada en 'is_suspicious' de los puntos contenidos
# Truco: usamos el índice de fila terminal de cada ventana para recuperar MMSI y tiempo aproximado para análisis adicional
y_point = df[CFG["train_filter_col"]].fillna(0).astype(int).to_numpy()

# Para cada ventana, estimamos si contiene algún punto sospechoso dentro del rango [end-T+1, end]
# Como no guardamos los índices de todos los puntos, aproximamos con una rolling sobre la serie original
# Alternativa: reconstruir mejor el mapping índice->ventana si es necesario.
# Aquí, simplificamos: una ventana es "sospechosa" si el punto terminal (end) es sospechoso (proxy razonable).
y_win = y_point[idx_rows_all] if len(idx_rows_all) else np.array([], dtype=int)

print("Ventanas totales:", X_all.shape, "Sospechosas (proxy):", int(y_win.sum()))


In [ ]:

# Filtramos ventanas de entrenamiento: solo NORMAL (proxy)
train_mask = (y_win == 0)
X_train = X_all[train_mask]
groups_train = groups_all[train_mask]

# Para evaluación mantendremos todas las ventanas
X_eval = X_all
y_eval = y_win
groups_eval = groups_all

print("X_train:", X_train.shape, "X_eval:", X_eval.shape)

# Escalado: fit en TRAIN NORMAL, transform en ambos
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_sc = scaler.fit_transform(X_train)
X_eval_sc = scaler.transform(X_eval)

# Guardamos scaler
with open(os.path.join(CFG["out_dir"], f"{CFG['artifact_prefix']}_scaler.pkl"), "wb") as f:
    pickle.dump(scaler, f)


In [ ]:

# Buscamos hiperparámetros por pseudo-validación "contaminada" mínima:
# Al no tener negativos en train, usamos la estabilidad de scores y ratio de outliers producidos como heurística.
# Alternativa: usar un hold-out de ventanas con baja proporción (o 0) de sospechosos para early stopping.
param_grid = list(ParameterGrid({
    "nu": CFG["svm_nu_grid"],
    "gamma": CFG["svm_gamma_grid"]
}))

def outlier_rate(scores):
    # decision_function: mayores -> más normal. score_samples similar. Aquí usamos predicciones del modelo.
    # Usaremos 'predict' que devuelve +1 normal, -1 outlier.
    neg = (scores == -1).mean()
    return float(neg)

best_cfg = None
best_obj = None  # minimizamos desviación del target_outlier_rate (heurística) y varianza entre folds
target_outlier_rate = 0.05

gkf = GroupKFold(n_splits=min(CFG["kfold_splits"], len(np.unique(groups_train))))
results = []

for p in param_grid:
    fold_rates = []
    for tr_idx, va_idx in gkf.split(X_train_sc, groups=groups_train):
        Xtr, Xva = X_train_sc[tr_idx], X_train_sc[va_idx]
        model = OneClassSVM(kernel=CFG["kernel"], nu=p["nu"], gamma=p["gamma"])
        model.fit(Xtr)
        pred = model.predict(Xva)  # +1 normal, -1 outlier
        rate = outlier_rate(pred)
        fold_rates.append(rate)
    rate_mean = float(np.mean(fold_rates))
    rate_std = float(np.std(fold_rates))
    obj = abs(rate_mean - target_outlier_rate) + rate_std
    results.append({"params": p, "rate_mean": rate_mean, "rate_std": rate_std, "obj": obj})
    if best_obj is None or obj < best_obj:
        best_obj = obj
        best_cfg = p

res_df = pd.DataFrame(results).sort_values("obj")
print("Top 5 configs by heuristic objective:")
display(res_df.head(5))
print("Best params:", best_cfg)


In [ ]:

final_model = OneClassSVM(kernel=CFG["kernel"], nu=best_cfg["nu"], gamma=best_cfg["gamma"])
final_model.fit(X_train_sc)

# Scores en eval
# decision_function: valores altos = más normal. Convertimos a "anomaly_score" = -decision
decision = final_model.decision_function(X_eval_sc)
anomaly_score = -decision

# Curvas y métricas (necesita y_eval binaria; 1=sospechoso)
roc = roc_auc_score(y_eval, anomaly_score) if len(np.unique(y_eval)) > 1 else np.nan
ap = average_precision_score(y_eval, anomaly_score) if len(np.unique(y_eval)) > 1 else np.nan

prec, rec, thr = precision_recall_curve(y_eval, anomaly_score)
pr_auc = auc(rec, prec) if len(rec) > 1 else np.nan

print(f"ROC-AUC: {roc:.4f} | PR-AUC: {pr_auc:.4f} | AP: {ap:.4f}")

# Guardamos artefactos
with open(os.path.join(CFG["out_dir"], f"{CFG['artifact_prefix']}_model.pkl"), "wb") as f:
    pickle.dump(final_model, f)

with open(os.path.join(CFG["out_dir"], f"{CFG['artifact_prefix']}_config.json"), "w") as f:
    json.dump(CFG | {"best_params": best_cfg, "metrics": {"roc_auc": float(roc), "pr_auc": float(pr_auc), "ap": float(ap)}}, f, indent=2)

# Guardamos scores por ventana
out_df = pd.DataFrame({
    "anomaly_score": anomaly_score,
    "y_eval": y_eval.astype(int),
    "mmsi": groups_eval,
    "idx_end": idx_rows_all
})
out_path = os.path.join(CFG["out_dir"], f"{CFG['artifact_prefix']}_eval_scores.parquet")
out_df.to_parquet(out_path, index=False)
print("Saved:", out_path)


In [ ]:

# --- Plots ---
# 1) Precision-Recall curve
plt.figure()
plt.plot(rec, prec, linewidth=2)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall curve (Eval)")
plt.grid(True)
plt.show()

# 2) Score distributions por etiqueta
plt.figure()
plt.hist(anomaly_score[y_eval==0], bins=50, alpha=0.6, label="Normal (win)")
plt.hist(anomaly_score[y_eval==1], bins=50, alpha=0.6, label="Sospechosa (win)")
plt.xlabel("Anomaly score (−decision)")
plt.ylabel("Count")
plt.title("Distribución de scores por etiqueta (Eval)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Selección de umbral por top‑k (p.ej., 1% de ventanas más anómalas)
k_rate = 0.01
k = max(1, int(len(anomaly_score) * k_rate))
thr_k = np.partition(anomaly_score, -k)[-k]
pred_k = (anomaly_score >= thr_k).astype(int)  # 1 = anómala

# Métricas @k
tp = int(((pred_k==1) & (y_eval==1)).sum())
fp = int(((pred_k==1) & (y_eval==0)).sum())
fn = int(((pred_k==0) & (y_eval==1)).sum())
prec_k = tp / (tp + fp) if (tp+fp)>0 else 0.0
rec_k  = tp / (tp + fn) if (tp+fn)>0 else 0.0
f1_k   = 2*prec_k*rec_k/(prec_k+rec_k) if (prec_k+rec_k)>0 else 0.0

print(f"@k={k_rate*100:.1f}% -> P: {prec_k:.3f} | R: {rec_k:.3f} | F1: {f1_k:.3f}  (k={k})")

# Guardamos predicciones @k
out_topk = out_df.copy()
out_topk["pred_topk"] = pred_k
out_topk_path = os.path.join(CFG["out_dir"], f"{CFG['artifact_prefix']}_preds_topk.parquet")
out_topk.to_parquet(out_topk_path, index=False)
print("Saved:", out_topk_path)


In [ ]:

# Agregamos por mmsi para ver tasa de detección por trayectoria (porcentaje de ventanas anómalas)
agg = out_df.assign(anom=(anomaly_score>=thr_k).astype(int)).groupby("mmsi").agg(
    n_win=("anomaly_score", "size"),
    anom_win=("anom", "sum"),
    mean_score=("anomaly_score", "mean")
).reset_index()
agg["anom_rate"] = agg["anom_win"] / agg["n_win"]
display(agg.sort_values("anom_rate", ascending=False).head(10))

agg_path = os.path.join(CFG["out_dir"], f"{CFG['artifact_prefix']}_mmsi_agg.parquet")
agg.to_parquet(agg_path, index=False)
print("Saved:", agg_path)



## Cómo ejecutar con tus datos

1. Asegúrate de colocar tu **Parquet enriquecido** en `data/ais_enriched.parquet` (o ajusta `CFG["input_parquet"]`). Debe contener:
   - Identificador: `mmsi`
   - Tiempo: `timestamp` (timezone aware)
   - Variables base: `lat`, `lon`, `speed`, `course`, `depth`, `dist_coast_km`, `dist_port_km`
   - Etiqueta de validación: `is_suspicious` (0/1), solo usada para evaluación.

2. Ajusta tamaños de **ventana `T`** y `step` en `CFG`.

3. Ejecuta todas las celdas. Los artefactos se guardarán en `runs/ocsvm/`:
   - `*_scaler.pkl`, `*_model.pkl`, `*_config.json`
   - `*_eval_scores.parquet`, `*_preds_topk.parquet`, `*_mmsi_agg.parquet`

4. Opcional: Integra mapas (GeoPandas + shapely) para estudiar casos dentro/de fuera de la RMG y generar figuras para el informe.
